# 計算樣本類別數

In [1]:
with open('sample_name.txt','r') as f:
    name = f.read().split("\n")
    name_dict = {}
    for i in name:
        key, value = i.split(":")
        name_dict[key] = value
    number_of_samples = len(name_dict)

In [2]:
sample_face = []
for i in range(number_of_samples):
    sample_face.append('sample%s'%i+'_face/')

# 建立trainset, validationset,testset 資料夾

In [3]:
import os

if not os.path.exists('trainset'):
    os.mkdir('trainset')
for i in range(number_of_samples):
    if not os.path.exists('trainset/'+sample_face[i]):
        os.mkdir('trainset/'+sample_face[i])

In [4]:
if not os.path.exists('validationset'):
    os.mkdir('validationset')
for i in range(number_of_samples):
    if not os.path.exists('validationset/'+sample_face[i]):
        os.mkdir('validationset/'+sample_face[i])

In [5]:
 if not os.path.exists('testset'):
        os.mkdir('testset')
for i in range(number_of_samples):
    if not os.path.exists('testset/'+sample_face[i]):
        os.mkdir('testset/'+sample_face[i])

# 將數據分割成train, validation, test

In [6]:
for i in range(number_of_samples):
    locals()['sample%s'%i] = os.listdir(sample_face[i])

In [7]:
from sklearn.model_selection import train_test_split

for i in range(number_of_samples):
    locals()['sample%s'%i+'_train_validation'], locals()['sample%s'%i+'_test'] = \
    train_test_split(locals()['sample%s'%i], test_size = 0.2, random_state = 42)
    print('sample%s'%i+'_train_validation:',len(locals()['sample%s'%i+'_train_validation']),
          '\tsample%s'%i+'_test:',len(locals()['sample%s'%i+'_test']))

sample0_train_validation: 60 	sample0_test: 15
sample1_train_validation: 88 	sample1_test: 23
sample2_train_validation: 108 	sample2_test: 28


In [8]:
for i in range(number_of_samples):
    locals()['sample%s'%i+'_train'], locals()['sample%s'%i+'_validation'] = \
    train_test_split(locals()['sample%s'%i+'_train_validation'], test_size = 0.2, random_state = 42)
    print('sample%s'%i+'_train:',len(locals()['sample%s'%i+'_train']),
          '\tsample%s'%i+'_validation:',len(locals()['sample%s'%i+'_validation']))

sample0_train: 48 	sample0_validation: 12
sample1_train: 70 	sample1_validation: 18
sample2_train: 86 	sample2_validation: 22


# 複製分割好的圖檔到trainset

In [9]:
from shutil import copyfile

def copyFileToDst(dataset, datafolder, srcfolder):
    for f in dataset:
        src = srcfolder+ f
        dst = datafolder+srcfolder+f
        copyfile(src, dst)

In [10]:
for i in range(number_of_samples):
    copyFileToDst(locals()['sample%s'%i+'_train'], 'trainset/', sample_face[i])
    copyFileToDst(locals()['sample%s'%i+'_validation'], 'validationset/', sample_face[i])
    copyFileToDst(locals()['sample%s'%i+'_test'], 'testset/', sample_face[i])

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,      #x坐标保持不变，而对应的y坐标按比例发生平移
                                   zoom_range = 0.2,       #可以让图片在长或宽的方向进行放大
                                   horizontal_flip = True) #水平翻转操作
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

Using TensorFlow backend.


In [12]:
training_set = train_datagen.flow_from_directory('trainset/', target_size = (64, 64), batch_size = 10,
                                                 class_mode = 'categorical')
validation_set = validation_datagen.flow_from_directory('validationset/', target_size = (64, 64), batch_size = 10, 
                                                        class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('testset/', target_size = (64, 64), batch_size = 10, 
                                            class_mode = 'categorical')

Found 204 images belonging to 3 classes.
Found 52 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [13]:
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense

classifier = Sequential() # Initialising the CNN
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) # Convolution
classifier.add(MaxPooling2D(pool_size = (2, 2))) # Max Pooling
classifier.add(Conv2D(32, (3, 3), activation = 'relu')) # Convolution
classifier.add(MaxPooling2D(pool_size = (2, 2))) # Max Pooling
classifier.add(Flatten()) # Flattening
classifier.add(Dense(units = 128, activation = 'relu')) # Fully Connected
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = number_of_samples, activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

In [14]:
# try:
#     classifier.load_weights("SaveModel/face.h5")
#     print("載入模型成功!繼續訓練模型")
# except :    
#     print("載入模型失敗!開始訓練一個新模型")

In [15]:
history = classifier.fit_generator(training_set,
                                   nb_epoch=10,
                                   nb_val_samples=30,
                                   steps_per_epoch = 30,
                                   verbose = 1,
                                   validation_data = validation_set)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=30, verbose=1, validation_data=<keras.pre..., epochs=10, validation_steps=30)`
  


Epoch 1/10
30/30 [==============================] - 2s 81ms/step - loss: 0.5179 - acc: 0.7969 - val_loss: 0.0349 - val_acc: 0.9808
Epoch 2/10
30/30 [==============================] - 2s 58ms/step - loss: 0.0164 - acc: 1.0000 - val_loss: 0.0503 - val_acc: 0.9808
Epoch 3/10
30/30 [==============================] - 2s 61ms/step - loss: 0.1336 - acc: 0.9618 - val_loss: 0.0454 - val_acc: 0.9808
Epoch 4/10
30/30 [==============================] - 2s 59ms/step - loss: 0.0285 - acc: 0.9900 - val_loss: 0.0042 - val_acc: 1.0000
Epoch 5/10
30/30 [==============================] - 2s 64ms/step - loss: 0.0044 - acc: 1.0000 - val_loss: 0.0113 - val_acc: 1.0000
Epoch 6/10
30/30 [==============================] - 2s 62ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0127 - val_acc: 1.0000
Epoch 7/10
30/30 [==============================] - 2s 61ms/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 8/10
30/30 [==============================] - 2s 61ms/step - loss: 0.0026 - a

In [16]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(r'sample0_face\sample0_0.png', target_size= (64,64))
test_image = image.img_to_array(test_image)
test_image.shape

(64, 64, 3)

In [17]:
test_image = np.expand_dims(test_image, axis = 0)
type(test_image)
test_image.shape

(1, 64, 64, 3)

In [18]:
classifier.predict_classes(test_image)

array([0], dtype=int64)

In [19]:
name_dict['sample'+str(classifier.predict_classes(test_image)[0])] 

'luokuofeng'

In [20]:
if not os.path.exists('SaveModel'):
    os.mkdir('SaveModel')
classifier.save_weights("SaveModel/face.h5")
print("Saved model to disk")

Saved model to disk
